In [2]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder

import datetime

In [3]:
df = pd.read_csv('eplresults.csv')
df


,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,1993-94,1993-08-14T00:00:00Z,Arsenal,Coventry,0,3,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1993-94,1993-08-14T00:00:00Z,Aston Villa,QPR,4,1,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1993-94,1993-08-14T00:00:00Z,Chelsea,Blackburn,1,2,A,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1993-94,1993-08-14T00:00:00Z,Liverpool,Sheffield Weds,2,0,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1993-94,1993-08-14T00:00:00Z,Man City,Leeds,1,1,D,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10729,2020-21,2021-04-11T14:05:00Z,West Ham,Leicester,3,2,H,2.0,0.0,H,...,4.0,5.0,1.0,9.0,7.0,13.0,3.0,3.0,0.0,0.0
10730,2020-21,2021-04-11T16:30:00Z,Tottenham,Man United,1,3,A,1.0,0.0,H,...,3.0,7.0,7.0,5.0,11.0,15.0,1.0,5.0,0.0,0.0
10731,2020-21,2021-04-11T19:00:00Z,Sheffield United,Arsenal,0,3,A,0.0,1.0,A,...,2.0,5.0,2.0,4.0,9.0,6.0,1.0,1.0,0.0,0.0
10732,2020-21,2021-04-12T18:00:00Z,West Brom,Southampton,3,0,H,2.0,0.0,H,...,6.0,4.0,5.0,10.0,12.0,11.0,0.0,1.0,0.0,0.0


In [113]:
df = df.loc[2824:]
df.drop(['DateTime', 'Referee'], axis=1, inplace=True)

In [114]:
df = df.reset_index(drop=True)

In [115]:
df = pd.get_dummies(df, columns=['HomeTeam'], prefix = ['HomeTeam'])
df = pd.get_dummies(df, columns=['AwayTeam'], prefix = ['AwayTeam'])
df = pd.get_dummies(df, columns=['HTR'], prefix = ['HTR'])

df.head()

,Season,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,...,AwayTeam_Swansea,AwayTeam_Tottenham,AwayTeam_Watford,AwayTeam_West Brom,AwayTeam_West Ham,AwayTeam_Wigan,AwayTeam_Wolves,HTR_A,HTR_D,HTR_H
0,2000-01,4,0,H,2.0,0.0,17.0,8.0,14.0,4.0,...,0,0,0,0,0,0,0,0,0,1
1,2000-01,4,2,H,1.0,0.0,17.0,12.0,10.0,5.0,...,0,0,0,0,1,0,0,0,0,1
2,2000-01,1,3,A,1.0,1.0,6.0,16.0,3.0,9.0,...,0,0,0,0,0,0,0,0,1,0
3,2000-01,2,2,D,1.0,2.0,6.0,13.0,4.0,6.0,...,0,0,0,0,0,0,0,1,0,0
4,2000-01,2,0,H,2.0,0.0,17.0,12.0,8.0,6.0,...,0,0,0,0,0,0,0,0,0,1


In [116]:
label_encoder = LabelEncoder()


df['FTR']= label_encoder.fit_transform(df['FTR']) 
  
print('Unique values for our label are: ', df['FTR'].unique())
print('if the home team wins the label is ', df['FTR'][0])
print('if the away team wins the label is ', df['FTR'][2])
print('if there is a tie the label is ', df['FTR'][3])

label = df['FTR']
print('the result for the match in row 149 is ', label[149])

print(df.iloc[:,3:113])

features = df.iloc[:,3:113]

Unique values for our label are:  [2 0 1]
if the home team wins the label is  2
if the away team wins the label is  0
if there is a tie the label is  1
the result for the match in row 149 is  2
      FTR  HTHG  HTAG    HS    AS   HST  AST    HC    AC    HF  ...  \
0       2   2.0   0.0  17.0   8.0  14.0  4.0   6.0   6.0  13.0  ...   
1       2   1.0   0.0  17.0  12.0  10.0  5.0   7.0   7.0  19.0  ...   
2       0   1.0   1.0   6.0  16.0   3.0  9.0   8.0   4.0  15.0  ...   
3       1   1.0   2.0   6.0  13.0   4.0  6.0   5.0   8.0  11.0  ...   
4       2   2.0   0.0  17.0  12.0   8.0  6.0   6.0   4.0  21.0  ...   
...   ...   ...   ...   ...   ...   ...  ...   ...   ...   ...  ...   
7905    2   2.0   0.0   4.0  15.0   4.0  5.0   1.0   9.0   7.0  ...   
7906    0   1.0   0.0  12.0  12.0   3.0  7.0   7.0   5.0  11.0  ...   
7907    0   0.0   1.0   7.0  14.0   2.0  5.0   2.0   4.0   9.0  ...   
7908    2   2.0   0.0  15.0  14.0   6.0  4.0   5.0  10.0  12.0  ...   
7909    1   0.0   0.0  23

In [117]:
from sklearn.model_selection import train_test_split

y=np.ravel(label)
X = features


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

print("The shape of X_train is " + str(X_train.shape))
print("The size of y_train is " + str(y_train.shape))
print("The size of X_test set is " + str(X_test.shape))
print("The size of y_test is " + str(y_test.shape))

The shape of X_train is (5299, 104)
The size of y_train is (5299,)
The size of X_test set is (2611, 104)
The size of y_test is (2611,)
